In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from fastshap import ImageFastSHAP, ShapleySampler, ResizeMask

/home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-15 11:54:14.835083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 11:54:16.806268: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/mesa-diverted/x86_64-lin

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, Reshape, Multiply)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

In [4]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

## Load Data

In [6]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

In [7]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


2023-06-15 11:54:19.974369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-15 11:54:19.986588: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-15 11:54:19.987334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-15 11:54:19.988545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

### Batch Data

In [8]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [9]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Surrogate Imputer

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50

P = 14*14
value_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
) 
D = 10

model_input = Input(shape=INPUT_SHAPE, dtype='float64', name='input')
S = ShapleySampler(P, paired_sampling=False, num_samples=1)(model_input)
S = Lambda(lambda x: tf.cast(x, tf.float32))(S)
S = Reshape((P,))(S)
S = ResizeMask(in_shape=INPUT_SHAPE, mask_size=P)(S)
xs = Multiply()([model_input, S])

net = value_model(xs)
out = Dense(D, activation='softmax')(net)

surrogate = Model(model_input, out)

# Get Checkpointed Model
weights_path = 'surrogate/20230614_13_43_41/value_weights.h5'
surrogate.load_weights(weights_path)

# Remove Masking Layer
surrogate = Sequential(   
    [l for l in surrogate.layers[-2:]]
)
surrogate.trainable = False

Use alpha and beta
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


# Train FastSHAP

### Save Dir

In [11]:
date = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'fastshap_ws'
model_dir = os.path.join(os.getcwd(), save_dir, date)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Initialize

In [12]:
from importlib import reload
import fastshap
import utils
reload(fastshap)
reload(utils)
from fastshap import ImageFastSHAP

In [13]:
fastshap = ImageFastSHAP(imputer = surrogate,
                         normalization=None,
                         model_dir = model_dir, 
                         link='logit')

2023-06-15 11:54:29.766992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200


1/1 [==============================] - 6s 6s/step


### Train

In [14]:
t = time.time()
fastshap.train(train_data = ds_train, 
              val_data = ds_val, 
              max_epochs = EPOCHS, 
              batch_size = BATCH_SIZE, 
              num_samples = 1,
              lr = LR,
              paired_sampling = False, 
              eff_lambda = 0.0,
              verbose = 1,
              lookback = 10)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)

Use alpha and beta
The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'conv1_conv/kernel:0' shape=(7, 7, 3, 64) dtype=float32>
  <tf.Variable 'conv1_conv/bias:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1_bn/beta:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_0_conv/kernel:0' shape=(1, 1, 64, 256) dtype=float32>
  <tf.Variable 'conv2_block1_0_conv/bias:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_0_bn/gamma:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_0_bn/beta:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_1_conv/kernel:0' shape=(1, 1, 64, 64) dtype=float32>
  <tf.Variable 'conv2_block1_1_conv/bias:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_1_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_1_bn/beta:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_bloc

2023-06-15 11:54:55.749242: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55dbda278160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-15 11:54:55.749287: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-06-15 11:54:55.769689: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-15 11:54:56.531982: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1563/1563 [==============================] - ETA: 0s - loss: 137.5922 - shap_loss: 137.5922 - eff_loss: 0.0000e+00
Epoch 1: val_shap_loss improved from inf to 34.94696, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/fastshap/20230615_11_54_27/explainer_weights.h5
1563/1563 [==============================] - 716s 437ms/step - loss: 137.5922 - shap_loss: 137.5922 - eff_loss: 0.0000e+00 - val_loss: 34.9470 - val_shap_loss: 34.9470 - val_eff_loss: 0.0000e+00 - lr: 0.0010
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 29.3959 - shap_loss: 29.3959 - eff_loss: 0.0000e+00
Epoch 2: val_shap_loss improved from 34.94696 to 26.62273, saving model to /home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/fastshap/20230615_11_54_27/explainer_weights.h5
1563/1563 [==============================] - 679s 435ms/step - loss: 29.3959 - shap_loss: 29.3959 - eff_loss: 0.0000e+00 - val_loss: 26.6227 - val_shap_los

# Explain w/ FastSHAP

### Load Images

In [15]:
images_dir = os.path.join(os.getcwd(), 'images')
images = np.load(os.path.join(images_dir, 'processed_images.npy'), allow_pickle=True)

In [16]:
images.shape

(1000, 224, 224, 3)

### Explain

In [17]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

#from keras import backend as K
#tf.compat.v1.keras.backend.tensorflow_backend._get_available_gpus()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [18]:
t = time.time()
shap_values = fastshap.explainer.predict(images)
explaining_time = time.time() - t
shap_values = [shap_values[:,:,:,i] for i in range(10)]

32/32 [==============================] - 3s 63ms/step


2023-06-15 21:31:53.380999: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2007040000 exceeds 10% of free system memory.


### Save

In [19]:
model_dir

'/home/sidtandon/Sid/GitRepo/iclr-fastshap/fastshap/experiments/images/cifar10/fastshap/20230615_11_54_27'

In [20]:
# save_dir = 'fastshap'
# model_dir = os.path.join(os.getcwd(), save_dir, 'results')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)
    
with open(os.path.join(model_dir, 'shap_values.pkl'), 'wb') as f:
    pickle.dump(shap_values, f)